In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from utils import process_tweet
import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
import os

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rjaditya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
'''
1. split the data into train and test
2. create vocab
3. get word embeddings
4. train a NN
5. evaluate
'''

'\n1. split the data into train and test\n2. create vocab\n3. get word embeddings\n4. train a NN\n5. evaluate\n'

In [3]:
dataset_path = '/Users/rjaditya/Documents/NEU-SEM/Spring-2024/ML/LLM-vs-Human-Text-Detection/dataset/train_v2_drcat_02.csv'
df = pd.read_csv(dataset_path, header = 0)
### get the labels
label = df['label']
df.drop(columns='label', axis = 1, inplace = True)
df.drop(columns='RDizzl3_seven', axis = 1, inplace=True)

### remove a feature that is not important for the task
df.drop(columns='prompt_name', axis =1, inplace = True)

### one hot encode the source (human, llm1, llm2, ...)
encoded = pd.get_dummies(df['source'])
source = set(df['source'])
for i in source:
    encoded[i] = encoded[i].astype('int')
df = pd.concat([df, encoded], axis = 1)
df.drop('source', axis = 1, inplace=True)

### change the label
df.head()

,text,NousResearch/Llama-2-7b-chat-hf,chat_gpt_moth,cohere-command,darragh_claude_v6,darragh_claude_v7,falcon_180b_v1,kingki19_palm,llama2_chat,llama_70b_v1,mistral7binstruct_v1,mistral7binstruct_v2,mistralai/Mistral-7B-Instruct-v0.1,palm-text-bison1,persuade_corpus,radek_500,radekgpt4,train_essays
0,Phones\n\nModern humans today are always on th...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,This essay will explain if drivers should or s...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Driving while the use of cellular devices\n\nT...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Phones & Driving\n\nDrivers should not be able...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Cell Phone Operation While Driving\n\nThe abil...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [4]:
### Train test split (75, 25)
# X_train, Y_train, X_test, Y_test = train_test_split(df, label, shuffle = True)

In [5]:
from transformers import AlbertTokenizer, AlbertModel
import torch

name = 'albert-base-v2'
tokenizer = AlbertTokenizer.from_pretrained(name)
model = AlbertModel.from_pretrained(name)


/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def pre_process(essay):
    doc = process_tweet(essay)
    print(doc)
    return doc

def convert_essay_to_embeddings(essay):
    #preprocess = pre_process(essay) # if you want to stem and lemmatize your dataset. 
    tokens = tokenizer.encode(essay, padding=True , truncation=True, add_special_tokens = True, return_tensors = 'pt')
    with torch.no_grad():
        op = model(tokens)
        embeddings = op.last_hidden_state
    pooled_embedding = torch.mean(embeddings, dim = 1).squeeze()
    return pooled_embedding.numpy()

In [8]:
from tqdm import tqdm
embeddings = []
for essay in tqdm(list(df['text'])):
    embeddings.append(convert_essay_to_embeddings(essay))
    break
print(embeddings)

  0%|          | 0/44868 [00:00<?, ?it/s]

[array([ 1.74588963e-01, -5.08472502e-01,  1.05227172e+00, -5.46216145e-02,
        6.79007530e-01, -2.87307054e-01, -1.59669951e-01, -3.30062628e-01,
        4.32679713e-01,  4.56777096e-01,  6.03282869e-01,  4.38088834e-01,
       -3.52377743e-01, -1.75514057e-01,  4.23209935e-01, -8.63234028e-02,
       -6.01137280e-01, -2.31516764e-01,  1.05221227e-01, -3.50072801e-01,
       -3.44410203e-02, -8.35920870e-01,  2.42740571e-01,  7.41337478e-01,
       -2.59964198e-01,  3.66370976e-01,  4.99429941e-01, -9.81405750e-02,
       -1.03788882e-01,  4.67962027e-01, -4.48120594e-01, -2.85039563e-02,
       -8.74108553e-01,  3.05451125e-01, -9.17681408e+00, -5.07630587e-01,
       -2.42590383e-01,  6.80089235e-01,  3.96943480e-01, -2.81547576e-01,
        5.43361783e-01, -2.94654548e-01,  5.55241823e-01,  4.59043503e-01,
       -3.21817458e-01, -4.58050728e-01, -4.98489887e-01, -5.07370457e-02,
        1.73128083e-01,  7.57893860e-01,  2.74428785e-01,  4.22339410e-01,
        1.54102653e-01, 

In [ ]:
print(len(embeddings) == len(df['text']))
print("*****************************************")
print(embeddings[0])